In [ ]:
from modules.csv_processor import CSVProcessor
from modules.data_augmentation import DataAugmentation
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import csv
from sklearn.compose import ColumnTransformer

In [ ]:
processor = CSVProcessor("./Dataset")
X_train, y_train = processor.split()
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numerical_cols)])

# Fit and transform the training data
X_train_scaled = pd.DataFrame(preprocessor.fit_transform(X_train), columns=X_train.columns)

# Transform the test data
X_test_scaled = pd.DataFrame(preprocessor.transform(X_test), columns=X_test.columns)

In [ ]:
# 4. Split the Data into Training and Test Sets
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
data_augmentation = DataAugmentation(X_train, y_train, num_augmented_samples=3000)
X_train, y_train = data_augmentation.apply_data_augmentation()

In [ ]:
input_shape = X_train.shape[1]
num_classes = len(np.unique(y_train))

model = Sequential([
    Dense(64, activation='relu', input_shape=(input_shape,)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train, y_train, batch_size=4, epochs=100, verbose=1)

In [ ]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test)

print("Testing Accuracy:", accuracy)